In [11]:
#Importing...
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
#Load data
data = pd.read_csv('cleaned_data.csv')
data.head()

,Unnamed: 0,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,0,19,27.900,0,16884.92400,0,1,0,0,1
1,1,18,33.770,1,1725.55230,1,0,0,1,0
2,2,28,33.000,3,4449.46200,1,0,0,1,0
3,3,33,22.705,0,21984.47061,1,0,1,0,0
4,4,32,28.880,0,3866.85520,1,0,1,0,0


In [7]:
#Features and target variabbles

X = data.drop(columns=['charges'])
y = data['charges']

In [8]:
#Split data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [9]:
randomForestModel = RandomForestRegressor(n_estimators=100, random_state=42)
randomForestModel.fit(X_train, y_train)
predictions = randomForestModel.predict(X_test)

In [12]:
#Model Evaluation
mse = mean_squared_error(y_test, y_pred=predictions)

r2 = r2_score(y_test, y_pred=predictions)

print(f"Mean Squared Error: {mse}\n"
      f"R-squared Score: {r2}"
      )

Mean Squared Error: 20478073.23579169
R-squared Score: 0.8885583812286515
